In [ ]:
import pandas as pd
import numpy as np
import time
import sys
sys.path.append('tools/tensorflow-DeepFM-master/')

import config
import DataReader
from DeepFM import DeepFM




import tensorflow as tf

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import log_loss,roc_auc_score

<font color=#0099ff size=5 face="黑体">读取数据</font>

In [ ]:
dfAll = pd.read_table(config.FEATURE_SET_INPUT,sep=' ')
for var in dfAll:
    if dfAll[var].dtypes=='float':
        dfAll[var] = pd.to_numeric(dfAll[var],downcast = 'float')
    else:
        dfAll[var] = pd.to_numeric(dfAll[var],downcast='usigned')

<font color=#0099ff size=5 face="黑体">生成稀疏特征</font>

In [ ]:
dfTrain = dfAll.loc[(dfAll['hour']<12)&(dfAll['hour']>0)]
dfTest = dfAll.loc[dfAll['hour']>=12]

train_idx = dfTrain.loc[dfTrain['hour']<10].index
valid_idx = dfTrain.loc[dfTrain['hour']>=10].index
del dfAll

In [ ]:
fd = DataReader.FeatureDictionary(dfTrain=dfTrain, dfTest=dfTest,
                           numeric_cols=[i for i in dfTrain.columns],
                           ignore_cols=['is_trade','instance_id'],
                           active_bound=1)
data_parser = DataReader.DataParser(feat_dict=fd)
Xi_train, Xv_train, y_train = data_parser.parse(df=dfTrain, has_label=True)
Xi_test, Xv_test, ids_test = data_parser.parse(df=dfTest)

del dfTrain
del dfTest

<font color=#0099ff size=5 face="黑体">设置模型参数</font>

In [ ]:
dfm_params = {
    "use_fm": True,
    "use_deep": True,
    "embedding_size": 8,
    "dropout_fm": [1.0, 1.0],
    "deep_layers": [32, 32],
    "dropout_deep": [0.5, 0.5, 0.5],
    "deep_layers_activation": tf.nn.relu,
    "epoch": 20,
    "batch_size": 2048,
    "learning_rate": 0.001,
    "optimizer_type": "adam",
    "batch_norm": 1,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.01,
    "verbose": True,
    "eval_metric": log_loss,
    "random_seed": config.RANDOM_SEED
}
dfm_params["feature_size"] = fd.feat_dim
dfm_params["field_size"] = len(Xi_train[0])
print(dfm_params["feature_size"])

In [ ]:
_get = lambda x,l:[x[i] for i in l]
Xi_train_, Xv_train_, y_train_ = _get(Xi_train, train_idx), _get(Xv_train, train_idx), _get(y_train, train_idx)
Xi_valid_, Xv_valid_, y_valid_ = _get(Xi_train, valid_idx), _get(Xv_train, valid_idx), _get(y_train, valid_idx)

dfm = DeepFM(**dfm_params)
dfm.fit(Xi_train_, Xv_train_, y_train_, Xi_valid_, Xv_valid_, y_valid_)